# Exploration de la base de données

In [1]:
#!conda install pandas #dans la console
from datetime import date, timedelta
import os

import pandas as pd

In [2]:
# Racine des fichiers quotidiens
BASE_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv'

# Dates de diponibilité des fichiers
START_DATE = date(2020, 1, 22)
END_DATE = date(2020, 3, 27)

#Répertoire de sauvegarde des fichiers bruts
RAWFILES_DIR = '../data/raw/'
PROCESSED_DIR = '../data/processed/'

## Boucle de récupération des fichiers

In [55]:
delta = END_DATE - START_DATE       # as timedelta

for i in range(delta.days + 1):
    day = START_DATE + timedelta(days=i)
    day_label = day.strftime("%m-%d-%Y")
    
    
    #virus_df = pd.read_csv(BASE_URL.format(day_label), sep=",", parse_dates=["Last Update"])
    
    virus_df = pd.read_csv(BASE_URL.format(day_label), sep=',')
    
    # Files from 22 March have new column names
    if ('FIPS' in virus_df.columns):
        virus_df = pd.read_csv(BASE_URL.format(day_label), sep=',', parse_dates=['Last_Update'])
        virus_df.columns = ['FIPS', 'Admin2', 'Province/State', 'Country/Region', 'Last Update', 'Latitude', 
                            'Longitude', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Combined_Key']
    else:
        virus_df = pd.read_csv(BASE_URL.format(day_label), sep=',', parse_dates=['Last Update'])
    
    # In new files, Mainland China is now China 
    virus_df['Country/Region'] = virus_df['Country/Region'].replace('Mainland China', 'China')
    
    # In data 3 ways of writting for South Korea
    virus_df['Country/Region'] = virus_df['Country/Region'].replace('Korea, South', 'South Korea')
    virus_df['Country/Region'] = virus_df['Country/Region'].replace('Republic of Korea', 'South Korea')

    virus_df.to_csv(os.path.join(RAWFILES_DIR, day_label + '.csv'), index=False)
    #print(day_label)

In [56]:
virus_df.dtypes

FIPS                     float64
Admin2                    object
Province/State            object
Country/Region            object
Last Update       datetime64[ns]
Latitude                 float64
Longitude                float64
Confirmed                  int64
Deaths                     int64
Recovered                  int64
Active                     int64
Combined_Key              object
dtype: object

## Constitution de la table de référence lat/long

In [57]:
import glob

df_list = []

# Lecture des fichiers récupérés et sélection de ceux qui ont une lat / long
for file in glob.glob(os.path.join(RAWFILES_DIR, '*.csv')):
    virus_df = pd.read_csv(file, sep=',')
    if 'Latitude' in virus_df.columns and 'Longitude' in virus_df.columns:
        df_list.append(virus_df)

all_df = pd.concat(df_list)

# Table de référence pour les lat / long
(all_df[["Province/State", "Country/Region", "Latitude", "Longitude"]]
 # Delete where Latitude = 0 and Longitude = 0 for US and Canada
 .query('Latitude != 0 & Longitude != 0')
 .drop_duplicates(subset=["Province/State", "Country/Region"])
 .sort_values(by=["Country/Region", "Province/State"])
 .to_csv(os.path.join(PROCESSED_DIR, "lat_long_table.csv"), index=False)
)

In [58]:
(all_df[["Province/State", "Country/Region", "Latitude", "Longitude"]]
 .drop_duplicates()
 [(all_df[["Province/State", "Country/Region", "Latitude", "Longitude"]]
   .drop_duplicates()
   .duplicated(subset=["Province/State", "Country/Region"], keep=False))]
)
# Les 3 pays qui ont "bougé".

,Province/State,Country/Region,Latitude,Longitude
1,NaN,South Korea,36.000,128.0000
2,NaN,Italy,43.000,12.0000
4,Henan,China,33.882,113.6140
8,NaN,Iran,32.000,53.0000
15,Heilongjiang,China,47.862,127.7615
...,...,...,...,...
2858,California,US,0.000,0.0000
2860,Delaware,US,0.000,0.0000
2866,Maryland,US,0.000,0.0000
2870,Nebraska,US,0.000,0.0000


## Construction d'une table unique

In [59]:
data_catalog = {
    'Last Update':["<M8[ns]"],
    "Confirmed":["float64", "int64"],
    "Deaths":["float64", "int64"],
    "Recovered":["float64", "int64"],
    "Latitude":["float64"],
    "Longitude":["float64"]
}

In [63]:
df_list = []

latlong_df = pd.read_csv(os.path.join(PROCESSED_DIR, "lat_long_table.csv"))

# Lecture des fichiers récupérés et sélection de ceux qui ont une lat / long
for file in glob.glob(os.path.join(RAWFILES_DIR, '*.csv')):
    virus_df = pd.read_csv(file, sep=',', parse_dates=["Last Update"])
    if not('Latitude' in virus_df.columns and 'Longitude' in virus_df.columns):
        virus_df = virus_df.merge(latlong_df, on=["Province/State", "Country/Region"], how='left')
    
    # Checker le type des variables dans l'importation de chaque fichier.
    for field, types in data_catalog.items():
        assert virus_df[field].dtypes in types, f"bad type for {field} in {file}"

    df_list.append(virus_df.assign(source=os.path.basename(file)))
 
all_df = pd.concat(df_list)

# Sauvegarde de la table totale
all_df.to_csv(os.path.join(PROCESSED_DIR, 'all_data.csv'), index=False)